In [2]:
import pandas as pd
import numpy as np

In [3]:
com = pd.read_csv('commodity.csv')

In [9]:
com

,date,ticker,price
0,2016-01-04,CF601,11845.0
1,2016-01-04,CF603,11500.0
2,2016-01-04,CF605,11305.0
3,2016-01-04,CF607,11225.0
4,2016-01-04,CF609,11225.0
...,...,...,...
194974,2021-03-24,ZC111,662.8
194975,2021-03-24,ZC112,656.4
194976,2021-03-24,ZC201,652.6
194977,2021-03-24,ZC202,650.6


In [5]:
del com['Unnamed: 0']

In [6]:
price = []
for i in list(com['price']):
    price.append(i.replace(',',''))

In [7]:
com['price'] = price

In [8]:
com['price'] = com['price'].astype(float)

In [10]:
#商品期货
def ema_12(x):
    x['ema_12'] = x['price'].rolling(12).mean()
    return x
def ema_26(x):
    x['ema_26'] = x['price'].rolling(26).mean()
    return x
def dif(x):
    x['dif'] = x['ema_12']-x['ema_26']
    return x
def macd(x):
    x['dea'] = x['dif'].rolling(9).mean()
    x['macd'] = x['dif']-x['dea']
    return x
def ma5(x):
    x['5'] = x['price'].rolling(5).mean()
    x['ma5'] = x['price']-x['5']
    return x
com = com.groupby('ticker').apply(lambda x: ema_12(x))
com = com.groupby('ticker').apply(lambda x: ema_26(x))
com = com.groupby('ticker').apply(lambda x: dif(x))
com = com.groupby('ticker').apply(lambda x: macd(x))
com = com.groupby('ticker').apply(lambda x: ma5(x))
com['macd_score'] = np.sign(com['macd'])
com['ma5_score'] = np.sign(com['ma5'])

In [11]:
com['date'] = pd.to_datetime(com['date'])

In [12]:
com

,date,ticker,price,ema_12,ema_26,dif,dea,macd,5,ma5,macd_score,ma5_score
0,2016-01-04,CF601,11845.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-04,CF603,11500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-04,CF605,11305.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-04,CF607,11225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-04,CF609,11225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
194974,2021-03-24,ZC111,662.8,647.633333,633.292308,14.341026,16.077635,-1.736610,660.64,2.16,-1.0,1.0
194975,2021-03-24,ZC112,656.4,645.133333,631.376923,13.756410,15.555271,-1.798860,656.76,-0.36,-1.0,-1.0
194976,2021-03-24,ZC201,652.6,642.966667,630.923077,12.043590,15.294587,-3.250997,653.96,-1.36,-1.0,-1.0
194977,2021-03-24,ZC202,650.6,641.783333,630.461538,11.321795,NaN,NaN,650.16,0.44,NaN,1.0


In [31]:
import datetime
import time

In [60]:
def backtest_com(start_date,end_date,universe,money,length):
    start_date_ = datetime.datetime.strptime(start_date,'%Y-%m-%d')
    end_date_ = datetime.datetime.strptime(end_date,'%Y-%m-%d')
    data = universe[(universe['date'] > start_date) & (universe['date'] < end_date)]
    value = [money]
    holding_tick = {}
    holding_vol = {}
    working_date = list(data.date.drop_duplicates())
    i = 0
    m = 0
    while i < len(working_date)-1:
        date = datetime.datetime.strptime(str(working_date[i])[0:10],'%Y-%m-%d')
        print(working_date[i+1])
        now = data[data['date']==str(date)[0:10]]
        date_1 = datetime.datetime.strptime(str(working_date[i+1])[0:10],'%Y-%m-%d')
        then = data[data['date']==str(date_1)[0:10]]
        pool = now[(now['macd_score']==1) & (now['ma5_score']==1)]
        pool = pool.sort_values(by='dif',ascending=False)
        holding_tick[str(date)[0:10]] = list(pool.ticker.drop_duplicates())[0:10]
        holding = {}
        whole = 0
        ticker_list = []
        for tick in holding_tick[str(date)[0:10]]:
            if tick in list(then.ticker.drop_duplicates()):
                ticker_list.append(tick)
            else:
                continue
        for ticker in ticker_list:
            print(ticker)
            holding[ticker] = value[m]/(len(ticker_list)*pool[pool['ticker']==ticker]['price'])
            date_ = datetime.datetime.strptime(str(working_date[i+1])[0:10],'%Y-%m-%d')
            now_ = data[data['date']==str(date_)[0:10]]
            whole = whole + (list(holding[ticker])[0] * list(now_[now_['ticker']==ticker]['price'])[0])
        holding_vol[date] = holding
        value.append(whole)
        m = m+1
        i = i+length
    #可能会有其他指标的计算，比如年化收益、sharpe ratio等等
    return holding_tick,holding_vol,value

In [67]:
value = backtest_com('2016-03-21','2021-03-01',com,100000000,1)[2]

2016-03-23 00:00:00
SM701
SM609
SM610
SM612
SM611
SF609
SF612
SF701
SF610
SF611
2016-03-24 00:00:00
SM701
SM609
SF609
TA610
TA611
TA701
TA612
OI701
TA607
OI609
2016-03-25 00:00:00
OI607
SR705
SR701
SR703
SR707
SR611
RM609
RM611
RM701
RM608
2016-03-28 00:00:00
SR705
SR703
SR707
SR611
RM609
RM701
RM611
RS609
RM608
RM607
2016-03-29 00:00:00
OI701
OI609
SR705
OI605
SR701
SR703
SR611
SR609
RM701
RM611
2016-03-30 00:00:00
OI607
SR705
SR701
SR703
SR607
RM608
RM607
WH609
WH611
WH605
2016-03-31 00:00:00
SR703
CF605
WH605
2016-04-01 00:00:00
CF605
WH605
2016-04-05 00:00:00
OI701
OI609
OI611
CF607
SR705
OI607
SR703
SR611
CF605
OI605
2016-04-06 00:00:00
OI701
OI611
OI609
CF607
SR705
CF609
OI607
CF701
SR701
SR707
2016-04-07 00:00:00
OI611
CF607
SR705
SR707
SR703
SR701
CF609
SR611
SR609
CF605
2016-04-08 00:00:00
SR705
SR707
SR703
SR701
SR611
CF609
SR609
SR607
RM611
SR605
2016-04-11 00:00:00
SR707
WH605
WH609
WH701
WH611
WH607
RI609
2016-04-12 00:00:00
WH605
WH609
WH701
WH611
WH607
RI609
2016-04-13 0

In [68]:
value

[100000000,
 99859560.56907693,
 97913936.2678189,
 98338071.51333395,
 97993346.6179628,
 97721586.55150004,
 97400579.7766042,
 97445516.00528897,
 97530752.89427763,
 97902300.89002874,
 97738372.05489592,
 97685704.73029421,
 97249353.61701593,
 97325621.55364777,
 96903566.84593265,
 97859165.03642537,
 99108739.51809944,
 98557289.41730872,
 99727098.38006184,
 102019077.30328278,
 105095941.7454343,
 107285307.90208192,
 108096817.05309051,
 107488210.16801688,
 108647670.65754518,
 107943212.64019701,
 107265471.62847276,
 109236801.02048063,
 108422948.66107099,
 107688166.47327498,
 108645012.14130251,
 107524405.94549337,
 108568920.28288533,
 108527919.93534647,
 108055918.1249814,
 108149675.97151065,
 108149675.97151066,
 108149675.97151066,
 108149675.97151066,
 107700460.9236433,
 108143087.21248259,
 107406751.43122733,
 108378548.83754174,
 105092948.08285965,
 103551995.76492916,
 103832751.288919,
 104774797.87791951,
 105872384.44237348,
 107468519.30394727,
 10608

In [69]:
result = pd.DataFrame()

In [70]:
result['value'] = value

In [ ]:
result.to_csv('result.csv')